I want to tell a small story regarding this post. 

For a long time, I wanted to do some sort of "le data analysis". My fingers were always itching to hit the keyboard, but anyone who knows me, knows I'm not an "idea guy". That is because whenever I try to use the right part of my brain (the [part of brain](http://brainmadesimple.com/left-and-right-hemispheres.html) associated with creativity and the arts), I go into an analysis paralysis phase. I end up trying to solve "creative" problems in the same way that I use for solving "logical" problems. Anyway, I'm getting better at this.

I racked my brain, trying to come up with an "idea". The problem was, that I was looking for something perfect, and too lazy to start the work without knowing that it will have the expected outcome (of gaining an audience and recognition). A wise co worker once told me that once you start working on something, ideas are bound to arise, based on whatever you have done. It all seemed to make sense, but its only recenyly that my mind has been able to fully accept this concept.

I was pretty sure I wanted to do something with the FB API. I feel that understanding human behaviour is one of the most interesting aspects of data analytics/science. Seeing the behaviour of a person in real life, then quantifying it and seeing the same behaviour in mathematical equations and graphs, well, there's just something orgasmic about that (I use a lot of sex related metaphors to describe programming/maths/music/scenic places, you might want to get used to that if you'll be reading my blog).

So, after months and months of "thinking my brain cells away", I finally have a shitty story to tell.

Ok, this one starts off by making a word cloud of all my status messages. Yes, all 702 of them, starting off with sometime in 2007 or so.

For the tech savvy folks, here's a little bit of juice on how to set this up yourself. You'll have to get an access token and sign up for an FB developer account:

In [ ]:
from wordcloud import WordCloud
import facebook

graph = facebook.GraphAPI(access_token='youraccesstoken', version='2.8')
posts = graph.get_all_connections(id='me', connection_name='posts')
all_status_text = ''

for post in posts:
    post_id = post['id']
    #get the text of the post
    try:
        status = post['message']
        all_status_text += (status + " ")
    except:
        continue

wordcloud = WordCloud().generate(all_status_text)
image = wordcloud.to_image()
image.show()

And this is what it looks like :

![Word cloud](wordcloud-12july-11pm.png)